# Data Pre-processing Phase
In this notebook I'm running the data pre-processing phase on the **Uber Fare Prices Dataset**. This dataset is available on a <a href="https://www.kaggle.com/datasets/yasserh/uber-fares-dataset" target="blank">Kaggle</a>. In this dataset, there are more than 200K observations with a total of 9 columns.

<img src="https://i.ibb.co/X7M4SK5/Screenshot-from-2022-07-01-10-12-49.png" width=900 />

The columns on the dataset are:
+ **key**                 -> an unique identifier for each trip (useless for modeling)
+ **fare_amout**          -> the cost of each trip in USD (target variable)
+ **pickup_datetime**     -> the precime datetime when the passenger was picked up
+ **passenger_count**     -> the quantity of passengers
+ **pickup_longitude**    -> the longitude where the trip began
+ **pickup_latitude**     -> the latitude where the trip began
+ **dropoff_longitude**   -> the longitude where the trip ended
+ **dropoff_latitude**    -> the latitude where the trip ended

***
This project is meant to be part of my personal portfolio.

**Author**: Arthur G.

## Importing Dependencies
Here I'm importing all the dependencies for this notebook.

In [1]:
# libs
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from src.features.build_features import check_holiday
from src.features.build_features import apply_heversine

# settings
seed = np.random.seed(42)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

## Loading Data
Now I'll load the raw dataset.

In [2]:
raw_df = pd.read_csv(os.path.join('..', 'data', 'raw', 'uber-drive-fare-data.csv'))
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         200000 non-null  int64  
 1   key                200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


## Data Pre-processing
In this section I'll by running all the pre-processing steps to make this data ready for analysis and machine learning modeling.

### Removing Useless Columns
Let's start off by removing useless columns.

In [3]:
raw_df = raw_df.drop(columns=['Unnamed: 0', 'key'])
raw_df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-74.000,40.738,-74.000,40.723,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994,40.728,-73.995,40.750,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005,40.741,-73.963,40.773,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976,40.791,-73.965,40.803,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925,40.744,-73.973,40.761,5


### Extracting Date Info
We can't use date variables in machine learning algorithms, specially in tabula data, so I'll be converting date from string to datetime and the extracting year, month, day of the year, day of the week and hour.

In [4]:
# converting dtype
raw_df.pickup_datetime = pd.to_datetime(raw_df.pickup_datetime)

# extracting info
raw_df['pickup_year'] = raw_df.pickup_datetime.map(lambda x: x.year)
raw_df['pickup_month'] = raw_df.pickup_datetime.map(lambda x: x.month)
raw_df['pickup_dayofyear'] = raw_df.pickup_datetime.map(lambda x: x.dayofyear)
raw_df['pickup_dayofweek'] = raw_df.pickup_datetime.map(lambda x: x.dayofweek)
raw_df['pickup_hour'] = raw_df.pickup_datetime.map(lambda x: x.hour)
raw_df['is_holiday'] = raw_df.pickup_datetime.map(lambda x: check_holiday(str(x.date())))

# removing the original date column
raw_df = raw_df.drop(columns=['pickup_datetime'])

raw_df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_dayofyear,pickup_dayofweek,pickup_hour,is_holiday
0,7.5,-74.000,40.738,-74.000,40.723,1,2015,5,127,3,19,0
1,7.7,-73.994,40.728,-73.995,40.750,1,2009,7,198,4,20,0
2,12.9,-74.005,40.741,-73.963,40.773,1,2009,8,236,0,21,0
3,5.3,-73.976,40.791,-73.965,40.803,3,2009,6,177,4,8,0
4,16.0,-73.925,40.744,-73.973,40.761,5,2014,8,240,3,17,0


### Removing Missing Data
Based on the information of the dataset I extracted at the beginning of this notebook, we know that there is only one row with missing data, so let's get rid of this one.

In [5]:
# removing explicit nans
raw_df = raw_df.dropna()

# removing missing lat long information (filled with 0.0)
raw_df = raw_df.drop(index=raw_df.query("pickup_latitude == 0").index)

# removing target var missing data
raw_df = raw_df.drop(index=raw_df.query("fare_amount == 0").index)
raw_df.reset_index(drop=True, inplace=True)

### Calculating Distance Between Two Coordinates (Lat, Long)
The Heversin function is a mathematical formula to calculate the distance between two pairs of coordinates on a sphere.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Law-of-haversines.svg/220px-Law-of-haversines.svg.png" width="200" />

Head over to it's wikipedia <a href="https://en.wikipedia.org/wiki/Haversine_formula" target="">page</a> for a deep understanding. I wrote a function implementing the heversin formula and I'm using it below.

In [6]:
raw_df['trip_distance_km'] = apply_heversine(
    raw_df.pickup_latitude,
    raw_df.pickup_longitude,
    raw_df.dropoff_latitude,
    raw_df.dropoff_longitude
)

### Reorganizing Columns
Before saving our processed dataset, let's put the columns in a more meaningful order.

In [7]:
raw_df = raw_df[
    [
        'pickup_year', 'pickup_month', 'pickup_dayofyear', 
        'pickup_dayofweek', 'pickup_hour', 'is_holiday', 
        'passenger_count', 'pickup_latitude', 'pickup_longitude',
        'dropoff_latitude' ,'dropoff_longitude', 'trip_distance_km', 'fare_amount'
    ]
]

raw_df.head()

,pickup_year,pickup_month,pickup_dayofyear,pickup_dayofweek,pickup_hour,is_holiday,passenger_count,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_distance_km,fare_amount
0,2015,5,127,3,19,0,1,40.738,-74.000,40.723,-74.000,1.683,7.5
1,2009,7,198,4,20,0,1,40.728,-73.994,40.750,-73.995,2.458,7.7
2,2009,8,236,0,21,0,1,40.741,-74.005,40.773,-73.963,5.036,12.9
3,2009,6,177,4,8,0,3,40.791,-73.976,40.803,-73.965,1.662,5.3
4,2014,8,240,3,17,0,5,40.744,-73.925,40.761,-73.973,4.475,16.0


## Data Serialization
Now it's time to save our processed dataset. Let's start by splitting a train and a test set for model training and validation in both research and production environments.

In [9]:
# splitting predictors and target variables
predictors = raw_df.drop(columns=['fare_amount'])
target = raw_df.fare_amount

# train-test split
x_train, x_test, y_train, y_test = train_test_split(
    predictors, 
    target, 
    test_size=0.3,
    random_state=seed
)

# assembling dataframes
train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)
train.columns = raw_df.columns
test.columns = raw_df.columns

train.head()

,pickup_year,pickup_month,pickup_dayofyear,pickup_dayofweek,pickup_hour,is_holiday,passenger_count,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_distance_km,fare_amount
178724,2014,6,176,2,17,0,1,40.789,-73.971,40.779,-73.978,1.265,6.5
119447,2009,7,187,0,14,0,1,40.746,-73.991,40.768,-73.957,3.767,12.1
53066,2014,11,321,0,9,0,1,40.764,-73.959,40.756,-73.975,1.622,13.0
48470,2013,12,347,4,8,0,1,40.733,-73.996,40.759,-73.974,3.498,12.5
153968,2011,11,318,0,21,0,1,40.729,-73.984,40.762,-73.992,3.692,10.9


Now let's serialize the train and test sets as csv files.

In [10]:
train.to_csv(os.path.join('..', 'data', 'finalized', 'train.csv'), index=False)
test.to_csv(os.path.join('..', 'data', 'finalized', 'test.csv'), index=False)

This finishes our data preprocessing phase.